# 送样任务完成表

## 1）issue

[订单信息-简道云触发器](https://dev.eastwestec.com/issues/3055)

## 2）分支

weld_backend: `feature/3055_orders_sync`  
switch: `feature/3055_orders_sync`  

## 3）服务

### 3.1）weld_backend 与 switch 服务启动

### 3.2）celery

weld_backend: `celery -A celery_app.celery worker`  
switch: `celery -A celery_app.celery worker`  

## 4）代码解析

### 4.1）weld_backend 创建定时任务：定时触发 switch 查询 jdy《送样任务完成表》待更新数据。
   `bapi.modules.order.tasks.timing_update_send_sample`
   > `bapi.services.middlewares.api.jiandaoyun.order.models.Order#search_info`  # 该任务触发简道云的接口  
   > `bapi.services.middlewares.client.JiandaoyunMixClient` 的 `order` 类属性注册上述的接口
   
### 4.2）switch 创建 portal 接口：响应查询简道云 并 将待更新数据 post 至 cecp 接口。  
   `mapi.modules.portal.jiandaoyun.order.views.search_send_sample_task`

### 4.3）weld_backend 创建 view 接口：响应查询 order 表单相关数据 并 触发 switch 更新 jdy。 
   `bapi.modules.order.views.OrderAPIView#send_sample`

### 4.4）switch 创建 portal 接口：更新 jdy《送样任务完成表》。
   `mapi.modules.portal.jiandaoyun.order.views.sync_send_sample_task`

## 5）测试

cecp / switch 可开启 debug 模式。

#### 方式一、jdy 《送样任务完成表》新建行项目，订单号必填；cecp 手动发起同步请求。

```python 
from bapi.modules.order.tasks import timing_update_send_sample
timing_update_send_sample()
```

---

# 2019.07.18 第一次优化

## 1）创建待导入的测试数据

```python
for obj in Order.query.yield_per(100):
	i += 1
	print obj.platform_order_number
	if i > 300:
		break
```

- 复制黏贴 至 《送样任务完成表_测试环境-数据模板.xlsx》'订单号' 字段，注意：'重置条件-按日重置' 字段必填。
